In [1]:
import os
os.chdir('../../task1_title/')


import json
from glob import glob

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from utils import extract_wrong_ratio, select_wrong_case_topN

def extract_results_by_method(method):
    test_results = glob(f'./saved_model-direct_exp/{method}/*/results_test*')
    test_results = [r for r in test_results if method in r]
    
    total_df = pd.DataFrame()

    for result_path in test_results:
        name = result_path.split('/')[-2]
        r = json.load(open(result_path,'r'))

        df = pd.DataFrame({
            'Metrics' : [m.upper() for m in list(r.keys())], 
            name      : list(r.values())
        })

        df = df.round(3)
        df = df[df.Metrics!='LOSS']
        df = df.set_index(keys=['Metrics']).T

        total_df = pd.concat([total_df, df],axis=0)

    order = [f + f'-{method}' for f in ['FNDNet_w_freeze_w2e', 'HAN_w_freeze_w2e', 'BTS']]
    total_df = total_df.loc[order]

    return total_df

def extract_wrong_sace_by_method(method):
    results_path = glob(f'./saved_model-direct_exp/{method}/*/exp_results_test.csv')

    results_path = [r for r in results_path if method in r]
    results_path.sort()
    results_dict = {}

    for i, path in enumerate(results_path):
        modelname = os.path.basename(os.path.dirname(path))
        results_dict[modelname] = pd.read_csv(path)

    for i, (modelname, df) in enumerate(results_dict.items()):

        wrong_ratio_df = extract_wrong_ratio(df)
        wrong_ratio_df.rename(columns={'wrong / total (%)':f'{modelname} wrong / total (%)'}, inplace=True)

        if i == 0:
            wrong_ratio_results = wrong_ratio_df
        else:
            wrong_ratio_results = pd.merge(
                wrong_ratio_results,
                wrong_ratio_df,
                on='category',
                how='inner'
            )
            
    return wrong_ratio_results

## 낚시(자동) / 낚시(직접) / 비낚시

# Metrics

In [2]:
print(extract_results_by_method(method='random_select').to_markdown())

|                                   |   AUROC |    F1 |   RECALL |   PRECISION |   ACC |
|:----------------------------------|--------:|------:|---------:|------------:|------:|
| FNDNet_w_freeze_w2e-random_select |   0.948 | 0.844 |    0.809 |       0.882 | 0.883 |
| HAN_w_freeze_w2e-random_select    |   0.984 | 0.927 |    0.902 |       0.953 | 0.944 |
| BTS-random_select                 |   1     | 0.997 |    0.998 |       0.996 | 0.997 |


In [3]:
print(extract_results_by_method(method='random_category_select').to_markdown())

|                                            |   AUROC |    F1 |   RECALL |   PRECISION |   ACC |
|:-------------------------------------------|--------:|------:|---------:|------------:|------:|
| FNDNet_w_freeze_w2e-random_category_select |   0.901 | 0.768 |    0.749 |       0.788 | 0.823 |
| HAN_w_freeze_w2e-random_category_select    |   0.967 | 0.881 |    0.858 |       0.906 | 0.909 |
| BTS-random_category_select                 |   1     | 0.994 |    0.995 |       0.993 | 0.995 |


In [4]:
print(extract_results_by_method(method='tfidf_title_category_select').to_markdown())

|                                                 |   AUROC |    F1 |   RECALL |   PRECISION |   ACC |
|:------------------------------------------------|--------:|------:|---------:|------------:|------:|
| FNDNet_w_freeze_w2e-tfidf_title_category_select |   0.8   | 0.636 |    0.587 |       0.693 | 0.737 |
| HAN_w_freeze_w2e-tfidf_title_category_select    |   0.835 | 0.683 |    0.653 |       0.717 | 0.763 |
| BTS-tfidf_title_category_select                 |   0.969 | 0.893 |    0.874 |       0.912 | 0.918 |


In [5]:
print(extract_results_by_method(method='tfidf_content_category_select').to_markdown())

|                                                   |   AUROC |    F1 |   RECALL |   PRECISION |   ACC |
|:--------------------------------------------------|--------:|------:|---------:|------------:|------:|
| FNDNet_w_freeze_w2e-tfidf_content_category_select |   0.79  | 0.631 |    0.595 |       0.672 | 0.728 |
| HAN_w_freeze_w2e-tfidf_content_category_select    |   0.809 | 0.649 |    0.61  |       0.692 | 0.741 |
| BTS-tfidf_content_category_select                 |   0.959 | 0.869 |    0.85  |       0.888 | 0.899 |


In [7]:
print(extract_results_by_method(method='sentence_embedding_title').to_markdown())

|                                              |   AUROC |    F1 |   RECALL |   PRECISION |   ACC |
|:---------------------------------------------|--------:|------:|---------:|------------:|------:|
| FNDNet_w_freeze_w2e-sentence_embedding_title |   0.783 | 0.647 |    0.648 |       0.646 | 0.723 |
| HAN_w_freeze_w2e-sentence_embedding_title    |   0.805 | 0.658 |    0.649 |       0.668 | 0.736 |
| BTS-sentence_embedding_title                 |   0.964 | 0.88  |    0.862 |       0.898 | 0.908 |


In [3]:
print(extract_results_by_method(method='sentence_embedding_contents').to_markdown())

|                                                 |   AUROC |    F1 |   RECALL |   PRECISION |   ACC |
|:------------------------------------------------|--------:|------:|---------:|------------:|------:|
| FNDNet_w_freeze_w2e-sentence_embedding_contents |   0.78  | 0.633 |    0.612 |       0.655 | 0.722 |
| HAN_w_freeze_w2e-sentence_embedding_contents    |   0.801 | 0.619 |    0.555 |       0.702 | 0.733 |
| BTS-sentence_embedding_contents                 |   0.967 | 0.887 |    0.869 |       0.904 | 0.913 |


In [4]:
print(extract_results_by_method(method='bow_title_category_select').to_markdown())

|                                               |   AUROC |    F1 |   RECALL |   PRECISION |   ACC |
|:----------------------------------------------|--------:|------:|---------:|------------:|------:|
| FNDNet_w_freeze_w2e-bow_title_category_select |   0.8   | 0.647 |    0.615 |       0.683 | 0.738 |
| HAN_w_freeze_w2e-bow_title_category_select    |   0.835 | 0.681 |    0.647 |       0.718 | 0.762 |
| BTS-bow_title_category_select                 |   0.97  | 0.896 |    0.881 |       0.912 | 0.92  |


In [6]:
print(extract_results_by_method(method='bow_content_category_select').to_markdown())

|                                                 |   AUROC |    F1 |   RECALL |   PRECISION |   ACC |
|:------------------------------------------------|--------:|------:|---------:|------------:|------:|
| FNDNet_w_freeze_w2e-bow_content_category_select |   0.786 | 0.641 |    0.627 |       0.656 | 0.725 |
| HAN_w_freeze_w2e-bow_content_category_select    |   0.804 | 0.63  |    0.572 |       0.701 | 0.737 |
| BTS-bow_content_category_select                 |   0.96  | 0.871 |    0.849 |       0.893 | 0.901 |


In [3]:
print(extract_results_by_method(method='ngram_title_category_select').to_markdown())

|                                                 |   AUROC |    F1 |   RECALL |   PRECISION |   ACC |
|:------------------------------------------------|--------:|------:|---------:|------------:|------:|
| FNDNet_w_freeze_w2e-ngram_title_category_select |   0.831 | 0.676 |    0.626 |       0.735 | 0.765 |
| HAN_w_freeze_w2e-ngram_title_category_select    |   0.861 | 0.708 |    0.661 |       0.763 | 0.787 |
| BTS-ngram_title_category_select                 |   0.977 | 0.912 |    0.886 |       0.938 | 0.933 |


In [4]:
print(extract_results_by_method(method='ngram_content_category_select').to_markdown())

|                                                   |   AUROC |    F1 |   RECALL |   PRECISION |   ACC |
|:--------------------------------------------------|--------:|------:|---------:|------------:|------:|
| FNDNet_w_freeze_w2e-ngram_content_category_select |   0.788 | 0.636 |    0.608 |       0.667 | 0.728 |
| HAN_w_freeze_w2e-ngram_content_category_select    |   0.805 | 0.644 |    0.615 |       0.677 | 0.734 |
| BTS-ngram_content_category_select                 |   0.961 | 0.874 |    0.845 |       0.905 | 0.905 |


# Wrong Case

In [5]:
print(extract_wrong_sace_by_method(method='random_select').to_markdown())

|    | category                     | BTS-random_select wrong / total (%)   | FNDNet_w_freeze_w2e-random_select wrong / total (%)   | HAN_w_freeze_w2e-random_select wrong / total (%)   |
|---:|:-----------------------------|:--------------------------------------|:------------------------------------------------------|:---------------------------------------------------|
|  0 | NonClickbait_Auto            | 139 / 48450 (0.29%)                   | 3361 / 48450 (6.94%)                                  | 1393 / 48450 (2.88%)                               |
|  1 | Clickbait_Auto_random_select | 72 / 31181 (0.23%)                    | 5956 / 31181 (19.10%)                                 | 3062 / 31181 (9.82%)                               |


In [6]:
print(extract_wrong_sace_by_method(method='random_category_select').to_markdown())

|    | category                              | BTS-random_category_select wrong / total (%)   | FNDNet_w_freeze_w2e-random_category_select wrong / total (%)   | HAN_w_freeze_w2e-random_category_select wrong / total (%)   |
|---:|:--------------------------------------|:-----------------------------------------------|:---------------------------------------------------------------|:------------------------------------------------------------|
|  0 | NonClickbait_Auto                     | 220 / 48450 (0.45%)                            | 6289 / 48450 (12.98%)                                          | 2785 / 48450 (5.75%)                                        |
|  1 | Clickbait_Auto_random_category_select | 151 / 31181 (0.48%)                            | 7815 / 31181 (25.06%)                                          | 4437 / 31181 (14.23%)                                       |


In [3]:
print(extract_wrong_sace_by_method(method='tfidf_title_category_select').to_markdown())

|    | category                             | BTS-tfidf_title_category_select wrong / total (%)   | FNDNet_w_freeze_w2e-tfidf_title_category_select wrong / total (%)   | HAN_w_freeze_w2e-tfidf_title_category_select wrong / total (%)   |
|---:|:-------------------------------------|:----------------------------------------------------|:--------------------------------------------------------------------|:-----------------------------------------------------------------|
|  0 | NonClickbait_Auto                    | 2623 / 48450 (5.41%)                                | 8099 / 48450 (16.72%)                                               | 8040 / 48450 (16.59%)                                            |
|  1 | Clickbait_Auto_tfidf_category_select | 3921 / 31181 (12.57%)                               | 12876 / 31181 (41.29%)                                              | 10828 / 31181 (34.73%)                                           |


In [6]:
print(extract_wrong_sace_by_method(method='tfidf_content_category_select').to_markdown())

|    | category                                     | BTS-tfidf_content_category_select wrong / total (%)   | FNDNet_w_freeze_w2e-tfidf_content_category_select wrong / total (%)   | HAN_w_freeze_w2e-tfidf_content_category_select wrong / total (%)   |
|---:|:---------------------------------------------|:------------------------------------------------------|:----------------------------------------------------------------------|:-------------------------------------------------------------------|
|  0 | NonClickbait_Auto                            | 3346 / 48450 (6.91%)                                  | 9039 / 48450 (18.66%)                                                 | 8476 / 48450 (17.49%)                                              |
|  1 | Clickbait_Auto_tfidf_content_category_select | 4673 / 31181 (14.99%)                                 | 12639 / 31181 (40.53%)                                                | 12152 / 31181 (38.97%)                                           

In [9]:
print(extract_wrong_sace_by_method(method='sentence_embedding_title').to_markdown())

|    | category                                | BTS-sentence_embedding_title wrong / total (%)   | FNDNet_w_freeze_w2e-sentence_embedding_title wrong / total (%)   | HAN_w_freeze_w2e-sentence_embedding_title wrong / total (%)   |
|---:|:----------------------------------------|:-------------------------------------------------|:-----------------------------------------------------------------|:--------------------------------------------------------------|
|  0 | NonClickbait_Auto                       | 3044 / 48450 (6.28%)                             | 11077 / 48450 (22.86%)                                           | 10047 / 48450 (20.74%)                                        |
|  1 | Clickbait_Auto_sentence_embedding_title | 4302 / 31181 (13.80%)                            | 10977 / 31181 (35.20%)                                           | 10947 / 31181 (35.11%)                                        |


In [5]:
print(extract_wrong_sace_by_method(method='bow_title_category_select').to_markdown())

|    | category                                 | BTS-bow_title_category_select wrong / total (%)   | FNDNet_w_freeze_w2e-bow_title_category_select wrong / total (%)   | HAN_w_freeze_w2e-bow_title_category_select wrong / total (%)   |
|---:|:-----------------------------------------|:--------------------------------------------------|:------------------------------------------------------------------|:---------------------------------------------------------------|
|  0 | NonClickbait_Auto                        | 2665 / 48450 (5.50%)                              | 8902 / 48450 (18.37%)                                             | 7938 / 48450 (16.38%)                                          |
|  1 | Clickbait_Auto_bow_title_category_select | 3726 / 31181 (11.95%)                             | 11996 / 31181 (38.47%)                                            | 10992 / 31181 (35.25%)                                         |


In [2]:
print(extract_wrong_sace_by_method(method='bow_content_category_select').to_markdown())

|    | category                                   | BTS-bow_content_category_select wrong / total (%)   | FNDNet_w_freeze_w2e-bow_content_category_select wrong / total (%)   | HAN_w_freeze_w2e-bow_content_category_select wrong / total (%)   |
|---:|:-------------------------------------------|:----------------------------------------------------|:--------------------------------------------------------------------|:-----------------------------------------------------------------|
|  0 | NonClickbait_Auto                          | 3157 / 48450 (6.52%)                                | 10247 / 48450 (21.15%)                                              | 7596 / 48450 (15.68%)                                            |
|  1 | Clickbait_Auto_bow_content_category_select | 4707 / 31181 (15.10%)                               | 11623 / 31181 (37.28%)                                              | 13332 / 31181 (42.76%)                                           |


In [3]:
print(extract_wrong_sace_by_method(method='ngram_title_category_select').to_markdown())

|    | category                                   | BTS-ngram_title_category_select wrong / total (%)   | FNDNet_w_freeze_w2e-ngram_title_category_select wrong / total (%)   | HAN_w_freeze_w2e-ngram_title_category_select wrong / total (%)   |
|---:|:-------------------------------------------|:----------------------------------------------------|:--------------------------------------------------------------------|:-----------------------------------------------------------------|
|  0 | NonClickbait_Auto                          | 1819 / 48450 (3.75%)                                | 7019 / 48450 (14.49%)                                               | 6398 / 48450 (13.21%)                                            |
|  1 | Clickbait_Auto_ngram_title_category_select | 3541 / 31181 (11.36%)                               | 11677 / 31181 (37.45%)                                              | 10581 / 31181 (33.93%)                                           |


In [4]:
print(extract_wrong_sace_by_method(method='ngram_content_category_select').to_markdown())

|    | category                                     | BTS-ngram_content_category_select wrong / total (%)   | FNDNet_w_freeze_w2e-ngram_content_category_select wrong / total (%)   | HAN_w_freeze_w2e-ngram_content_category_select wrong / total (%)   |
|---:|:---------------------------------------------|:------------------------------------------------------|:----------------------------------------------------------------------|:-------------------------------------------------------------------|
|  0 | NonClickbait_Auto                            | 2755 / 48450 (5.69%)                                  | 9457 / 48450 (19.52%)                                                 | 9175 / 48450 (18.94%)                                              |
|  1 | Clickbait_Auto_ngram_content_category_select | 4821 / 31181 (15.46%)                                 | 12227 / 31181 (39.21%)                                                | 11994 / 31181 (38.47%)                                           